# Demo 1: Foundation - Simple Conversational AI

This notebook demonstrates the evolution from basic completion to conversational AI using Azure OpenAI.

## What We'll Cover
1. **Setup** - Initialize Azure OpenAI client with secure authentication
2. **Single Question** - Ask about conferences to demonstrate basic capabilities
3. **Stateless Limitation** - Show how context is lost without conversation history
4. **Conversational Context** - Demonstrate how to maintain context across interactions


## 1. Setup: Azure OpenAI Client Configuration

Following Azure best practices, we'll use environment variables for secure configuration and set up proper error handling.

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import json

# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4.1-nano")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")

# Validate configuration
if not all([AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY]):
    raise ValueError("Missing required Azure OpenAI configuration. Please check your .env file.")

# Initialize Azure OpenAI client following Azure best practices
# Using secure credential management (environment variables, not hardcoded keys)
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

print("✅ Azure OpenAI client initialized successfully!")
print(f"📍 Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"🚀 Model Deployment: {AZURE_OPENAI_DEPLOYMENT}")
print(f"📅 API Version: {AZURE_OPENAI_API_VERSION}")

# Demo conference data from environment
DEMO_CONFERENCE = os.getenv("DEMO_CONFERENCE_NAME", "AI Innovation Summit 2025")
DEMO_VENUE = os.getenv("DEMO_VENUE", "Tech Convention Center")
DEMO_DATES = os.getenv("DEMO_DATES", "March 15-17, 2025")

print(f"\n🎯 Demo Context: {DEMO_CONFERENCE} at {DEMO_VENUE} ({DEMO_DATES})")

## 2. Single Question: Basic Chat Completion

Let's ask the AI about conference management to see its foundational capabilities. This demonstrates the basic completion functionality.

In [ ]:
# Helper function to call Azure OpenAI with error handling
def ask_ai(messages, show_full_response=False):
    """
    Send messages to Azure OpenAI and return the response.
    Includes proper error handling and retry logic following Azure best practices.
    """
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            messages=messages,
            max_tokens=1000,
            temperature=0.7
        )
        
        if show_full_response:
            print("🔍 Full API Response:")
            print(json.dumps(response.model_dump(), indent=2, default=str))
            print("\n" + "="*50 + "\n")
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"❌ Error calling Azure OpenAI: {str(e)}")
        return None

# First question about conference management
print("🎤 Question 1: What are some good conferences in Europe to attend as a tech professional?")
print("-" * 70)

messages = [
    {"role": "system", "content": "You are an expert conference management consultant with 15+ years of experience organizing large-scale events."},
    {"role": "user", "content": "What are some good conferences in Europe to attend as a tech professional?"}
]

response = ask_ai(messages)
if response:
    print("🤖 AI Response:")
    print(response)
else:
    print("Failed to get response from AI")

## 3. Follow-up Question WITHOUT Context: Demonstrating Statelessness

Now let's ask a follow-up question without providing the conversation history. This will show how the AI loses context between separate requests - a key limitation we need to solve.

In [ ]:
# Follow-up question WITHOUT conversation history
print("🎤 Follow-up Question (WITHOUT context): What would be the best conference to attend, given that I live in Belgium?")
print("-" * 70)

# Notice: We're starting fresh - no conversation history!
messages_without_context = [
    {"role": "system", "content": "You are an expert conference management consultant with 15+ years of experience organizing large-scale events."},
    {"role": "user", "content": "What would be the best conference to attend, given that I live in Belgium?"}
]

response_without_context = ask_ai(messages_without_context)
if response_without_context:
    print("🤖 AI Response (Without Context):")
    print(response_without_context)
    print("\n⚠️  Notice: The AI doesn't know what 'venue capacity issue' refers to!")
    print("   It has to ask for clarification or make assumptions.")
else:
    print("Failed to get response from AI")

## 4. Follow-up Question WITH Context: True Conversational AI

Now let's demonstrate proper conversational AI by maintaining the conversation history. This is the foundation for building intelligent, context-aware systems.

In [ ]:
# Follow-up question WITH full conversation history
print("🎤 Follow-up Question (WITH context): How can I solve the venue capacity issue?")
print("-" * 70)

# Build conversation history - this is the key to conversational AI!
conversation_history = [
    {"role": "system", "content": "You are an expert conference management consultant with 15+ years of experience organizing large-scale events."},
    {"role": "user", "content": "What are some good conferences in Europe to attend as a tech professional?"},
    {"role": "assistant", "content": response},  # Include the previous AI response
    {"role": "user", "content": "What would be the best conference to attend, given that I live in Belgium?"}  # Our follow-up question
]

response_with_context = ask_ai(conversation_history)
if response_with_context:
    print("🤖 AI Response (With Context):")
    print(response_with_context)
    print("\n✅ Notice: The AI now understands we're talking about conference venue capacity!")
    print("   It can provide specific, contextual solutions.")
else:
    print("Failed to get response from AI")

## 5. Conversation Summary: Key Learnings

Let's implement a simple conversation manager to demonstrate the evolution from stateless to stateful AI interactions.

In [ ]:
class SimpleConversationManager:
    """
    A basic conversation manager that maintains context across multiple interactions.
    This demonstrates the foundation for more sophisticated AI systems.
    """
    
    def __init__(self, system_prompt):
        self.messages = [{"role": "system", "content": system_prompt}]
        self.client = client
        self.deployment = AZURE_OPENAI_DEPLOYMENT
    
    def ask(self, user_message, show_history=False):
        """Ask a question and maintain conversation history"""
        # Add user message to history
        self.messages.append({"role": "user", "content": user_message})
        
        if show_history:
            print("📝 Current Conversation History:")
            for i, msg in enumerate(self.messages):
                role_emoji = {"system": "⚙️", "user": "👤", "assistant": "🤖"}
                print(f"   {i+1}. {role_emoji.get(msg['role'], '❓')} {msg['role']}: {msg['content'][:100]}...")
            print()
        
        try:
            # Get AI response
            response = self.client.chat.completions.create(
                model=self.deployment,
                messages=self.messages,
                max_tokens=1000,
                temperature=0.7
            )
            
            ai_response = response.choices[0].message.content
            
            # Add AI response to history
            self.messages.append({"role": "assistant", "content": ai_response})
            
            return ai_response
        
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            return None
    
    def get_conversation_length(self):
        """Get the number of exchanges in the conversation"""
        # Subtract 1 for system message, then divide by 2 (user + assistant pairs)
        return (len(self.messages) - 1) // 2

# Initialize conversation manager
conversation = SimpleConversationManager(
    "You are an expert conference management consultant specializing in large-scale technology events. "
    f"You're currently helping plan {DEMO_CONFERENCE} at {DEMO_VENUE}."
)

print("🎯 Interactive Conference Planning Session")
print("=" * 50)

# Demonstrate fluid conversation
questions = [
    "What's the most critical factor for attendee satisfaction at tech conferences?",
    "How would you measure that effectively?",
    "What if the budget is limited to $50k for those initiatives?"
]

for i, question in enumerate(questions, 1):
    print(f"\n🎤 Question {i}: {question}")
    print("-" * 70)
    
    response = conversation.ask(question, show_history=(i==1))  # Show history only for first question
    
    if response:
        print("🤖 AI Response:")
        print(response)
        print(f"\n📊 Conversation length: {conversation.get_conversation_length()} exchanges")
    else:
        print("Failed to get response")

print(f"\n✅ Final conversation contains {len(conversation.messages)} total messages")
print("   (1 system + multiple user/assistant pairs)")

## Key Takeaways: From Completion to Conversation

### What We Demonstrated

1. **🔧 Setup & Security**: Proper Azure OpenAI configuration using environment variables and secure credential management
2. **💬 Basic Completion**: Single-turn question answering with expert system prompting
3. **⚠️ Stateless Limitation**: How context is lost between separate API calls
4. **🔄 Conversational Context**: Maintaining conversation history for coherent multi-turn interactions
5. **🏗️ Architecture Foundation**: Simple conversation manager as building block for complex systems

*This foundation enables everything that follows!*